# Automating Hyperparameter Tuning with Keras Tuner

In [176]:
# finding the right optimizer for the model.
# finding the right number of nodes in the layer.
# finding the right number of layers.

In [177]:
import numpy as np
import pandas as pd

In [178]:
df = pd.read_csv("diabetes.csv")

In [179]:
# classification dataset

In [180]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [181]:
# what influence the outcome
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [182]:
# seperating x and y
X = df.iloc[:, :-1].values
y = df["Outcome"]

In [183]:
# scaling x

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [184]:
X = scaler.fit_transform(X)

In [185]:
# splitting data into train and test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# model building

In [186]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [187]:
model = Sequential()

model.add(Dense(32, activation="relu", input_dim = 8))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#  preparing the model to learn.

In [188]:
model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [189]:
# optimizer = "Adam" → tells the model how to adjust weights (smart gradient descent).

# loss = "binary_crossentropy" → used for binary classification (yes/no, 0/1).

# metrics = ["accuracy"] → show the accuracy while training.

# Start training the model 

In [190]:
model.fit(X_train, y_train, batch_size =32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5391 - loss: 0.7057 - val_accuracy: 0.7468 - val_loss: 0.6069
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7101 - loss: 0.6057 - val_accuracy: 0.7922 - val_loss: 0.5380
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7280 - loss: 0.5473 - val_accuracy: 0.8182 - val_loss: 0.5042
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7329 - loss: 0.5116 - val_accuracy: 0.8247 - val_loss: 0.4834
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7638 - loss: 0.4901 - val_accuracy: 0.7987 - val_loss: 0.4770
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7671 - loss: 0.4738 - val_accuracy: 0.7987 - val_loss: 0.4737
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7671 - loss: 0.4625 - val_accuracy: 0.7792 - val_loss: 0.4760
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7785 - loss: 0.4531 - val_accuracy: 0.7792 - 

# USING KERAS TUNER

In [191]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt


In [192]:
import kerastuner as kt

# 1. FINDING THE RIGHT OPTIMIZER

In [193]:
def build_model(hp): # hyperparameter

    # model building from scratch
    model = Sequential()
    model.add(Dense(32, activation="relu", input_dim = 8))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice("optimizer",values = ["adam","sgd", "rmsprop", "adadelta"])  # Try different optimizers to find the best one

    model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])

    return model

In [194]:

tuner = kt.RandomSearch(              # Create a RandomSearch tuner object
    build_model,                      # Use the build_model function to build models
    objective='val_accuracy',         # find the one with the best validation accuracy
    max_trials=5                      # Try up to 5 different sets of hyperparameters
)

Reloading Tuner from ./untitled_project/tuner0.json


In [195]:
tuner.search(X_train, y_train, epochs = 5, validation_data =(X_test, y_test))

In [196]:
# find the best optimizer out of the list given

tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [197]:
model = tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [198]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,377 (5.38 KB)

 Trainable params: 1,377 (5.38 KB)

 Non-trainable params: 0 (0.00 B)

In [199]:
model.fit(X_train, y_train, batch_size =32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7410 - loss: 0.4916 - val_accuracy: 0.7468 - val_loss: 0.5205
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7427 - loss: 0.4747 - val_accuracy: 0.7532 - val_loss: 0.5182
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7573 - loss: 0.4650 - val_accuracy: 0.7597 - val_loss: 0.5131
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7736 - loss: 0.4573 - val_accuracy: 0.7597 - val_loss: 0.5117
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7752 - loss: 0.4509 - val_accuracy: 0.7597 - val_loss: 0.5139
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7785 - loss: 0.4462 - val_accuracy: 0.7532 - val_loss: 0.5131
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7801 - loss: 0.4421 - val_accuracy: 0.7468 - val_loss: 0.5171
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7834 - loss: 0.4384 - val_accuracy: 0.7403 - v

# 1. FINDING THE RIGHT NUMBER OF NEURON

In [200]:

def build_model(hp):
    model = Sequential()
    
    # Tune number of neurons in hidden layer (from 8 to 120, step 5)
    units = hp.Int("units", min_value=8, max_value=120, step=5)
    
    model.add(Dense(units=units, activation="relu", input_dim=8))
    model.add(Dense(1, activation="sigmoid"))

    # Fixed optimizer
    model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=["accuracy"])
    
    return model    

In [201]:
# Create the tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite=True,
    
    # Save tuner search results in the folder 'my_dir/neuron_search' in the home.
    directory='my_dir',
    project_name='neuron_search'
)

In [202]:
# Start the search

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 06s


In [203]:
tuner.get_best_hyperparameters()[0].values

{'units': 73}

In [204]:
# Get the best trained model from the tuner (top 1 model)

model = tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [205]:
model.fit(X_train, y_train, batch_size =32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7655 - loss: 0.5150 - val_accuracy: 0.7922 - val_loss: 0.5110
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7736 - loss: 0.4919 - val_accuracy: 0.7792 - val_loss: 0.5057
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7720 - loss: 0.4791 - val_accuracy: 0.7662 - val_loss: 0.5030
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7720 - loss: 0.4700 - val_accuracy: 0.7727 - val_loss: 0.4996
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7704 - loss: 0.4637 - val_accuracy: 0.7792 - val_loss: 0.4992
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7671 - loss: 0.4590 - val_accuracy: 0.7727 - val_loss: 0.5014
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7720 - loss: 0.4541 - val_accuracy: 0.7662 - val_loss: 0.5023
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7785 - loss: 0.4513 - val_accuracy: 0.7792 - v

# FINDING THE RIGHT NUMBER OF LAYERS

In [206]:
def build_model(hp):

    model = Sequential()

    # Input layer
    model.add(Dense(83, activation="relu", input_dim=8))

    # Hidden layer
    for i in range(hp.Int("num_layers",min_value=1, max_value=15)):
        model.add(Dense(72, activation="relu"))
    # Add 1 to 15 hidden layers with 72 units each (number chosen by tuner)

        
    # output layer
    model.add(Dense(1, activation="sigmoid"))

    # Fixed optimizer
    model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=["accuracy"])
    
    return model    

    

In [207]:
# Create the tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite= True,  # avoid "already exists" error if re-running
    directory='my_dir',
    project_name='layer_search'
)

In [208]:
tuner.search(X_train, y_train, epochs = 5, validation_data =(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 09s


In [209]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [210]:
model.fit(X_train, y_train, batch_size =32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8176 - loss: 0.3798 - val_accuracy: 0.7403 - val_loss: 0.5688
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8176 - loss: 0.3801 - val_accuracy: 0.7338 - val_loss: 0.5696
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8241 - loss: 0.3802 - val_accuracy: 0.7403 - val_loss: 0.5674
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8192 - loss: 0.3785 - val_accuracy: 0.7403 - val_loss: 0.5676
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8208 - loss: 0.3784 - val_accuracy: 0.7403 - val_loss: 0.5683
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8241 - loss: 0.3781 - val_accuracy: 0.7403 - val_loss: 0.5678
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8192 - loss: 0.3775 - val_accuracy: 0.7403 - val_loss: 0.5739
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8274 - loss: 0.3778 - val_accuracy: 0.7338 - v

# now using Keras Tuner to automate the hyperparameters

In [211]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):
    model = Sequential()
    model.add(tf.keras.Input(shape=(8,)))

    # Tune number of hidden layers: 1 to 5 (reduced max layers)
    num_layers = hp.Int("num_layers", 1, 5)

    # Tune dropout rate once globally (0 or some rate)
    use_dropout = hp.Boolean("use_dropout")
    dropout_rate = hp.Float("dropout_rate", 0.1, 0.5, step=0.1) if use_dropout else 0.0

    # Tune L2 regularization strength
    l2_reg = hp.Float("l2_reg", 1e-5, 1e-2, sampling="log")

    for i in range(num_layers):
        units = hp.Int(f"units_{i}", 32, 128, step=8)  # smaller max units
        activation = hp.Choice(f"activation_{i}", ["relu", "tanh"])

        # Add Dense with L2 regularization
        model.add(Dense(units, activation=activation, kernel_regularizer=l2(l2_reg)))

        # Add batch normalization
        model.add(BatchNormalization())

        # Add dropout if enabled
        if use_dropout:
            model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation="sigmoid"))

    optimizer_choice = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])
    if optimizer_choice == "adam":
        lr = hp.Float("lr_adam", 1e-4, 1e-3, sampling="log")
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_choice == "sgd":
        lr = hp.Float("lr_sgd", 1e-4, 1e-3, sampling="log")
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    else:  # rmsprop
        lr = hp.Float("lr_rms", 1e-4, 1e-3, sampling="log")
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model


In [212]:
import tensorflow as tf
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    overwrite= True,  
    directory='my_dir',
    project_name='search'
)


In [213]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 12s


In [214]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'use_dropout': False,
 'l2_reg': 0.0033511793625188957,
 'units_0': 80,
 'activation_0': 'tanh',
 'optimizer': 'adam',
 'lr_adam': 0.000945444253937403,
 'dropout_rate': 0.2,
 'units_1': 112,
 'activation_1': 'relu',
 'units_2': 112,
 'activation_2': 'relu',
 'units_3': 56,
 'activation_3': 'tanh',
 'units_4': 120,
 'activation_4': 'relu',
 'lr_sgd': 0.0003123925273344148,
 'lr_rms': 0.0001571741464507939}

In [215]:
# Get the best model found by the tuner after hyperparameter tuning


model = tuner.get_best_models(num_models=1)[0]

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [216]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [217]:
model.fit(X_train, y_train, batch_size =32, epochs = 200, validation_data = (X_test, y_test),callbacks=[early_stopping])

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7622 - loss: 1.7193 - val_accuracy: 0.6948 - val_loss: 1.7913
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8029 - loss: 1.6258 - val_accuracy: 0.7078 - val_loss: 1.7455
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8322 - loss: 1.5378 - val_accuracy: 0.7143 - val_loss: 1.7095
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8306 - loss: 1.5085 - val_accuracy: 0.7143 - val_loss: 1.6938
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8420 - loss: 1.4714 - val_accuracy: 0.6883 - val_loss: 1.6817
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8404 - loss: 1.4564 - val_accuracy: 0.7208 - val_loss: 1.6460
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8485 - loss: 1.4202 - val_accuracy: 0.6883 - val_loss: 1.6394
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8502 - loss: 1.4098 - val_accuracy: 0.7208 - v